# Modelling

In [1]:
import tensorflow as tf
import keras
from keras.layers import Dense, Input, Lambda, Flatten, Layer
from keras import Sequential, Model, backend
import pandas as pd
import numpy as np

2022-10-20 13:58:35.348490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_train = pd.read_csv('/Users/chinmayasukumar/Documents/Springboard/Capstone #2/data/interim/X_train.csv')
y_train = pd.read_csv('/Users/chinmayasukumar/Documents/Springboard/Capstone #2/data/interim/y_train.csv')
X_test = pd.read_csv('/Users/chinmayasukumar/Documents/Springboard/Capstone #2/data/interim/X_test.csv')
y_test = pd.read_csv('/Users/chinmayasukumar/Documents/Springboard/Capstone #2/data/interim/y_test.csv')

In [3]:
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(),\
                                    y_train.to_numpy().reshape(X_train.shape[0],),\
                                    y_test.to_numpy().reshape(y_test.shape[0],)

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((432, 14), (432,), (186, 14), (186,))

In [5]:
type(X_train), type(y_train), type(X_test), type(y_test)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [6]:
encoder_inputs = keras.Input(shape=(X_train.shape[1],), name='input')

x = Dense(100, activation='relu', name='dense_1')(encoder_inputs)
x = Dense(100, activation='relu', name='dense_2')(x)
x = Dense(50, activation='relu', name='dense_3')(x)

z_mean = Dense(2, name='latent_mu')(x)
z_log_var = Dense(2, name='latent_log_var')(x) 

class Sampling(Layer):

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

2022-10-20 13:58:49.190928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 14)]         0           []                               
                                                                                                  
 dense_1 (Dense)                (None, 100)          1500        ['input[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 100)          10100       ['dense_1[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 50)           5050        ['dense_2[0][0]']                
                                                                                            

In [7]:
latent_inputs = keras.Input(shape=(2,), name='latent_inputs')

x = Dense(50, activation="relu", name='dense_4')(latent_inputs)
x = Dense(100, activation='relu', name='dense_5')(x)
x = Dense(100, activation='relu', name='dense_6')(x)

output_layer = Dense(X_train.shape[1], name='output')(x)


decoder = keras.Model(latent_inputs, output_layer, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 latent_inputs (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_4 (Dense)             (None, 50)                150       
                                                                 
 dense_5 (Dense)             (None, 100)               5100      
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 output (Dense)              (None, 14)                1414      
                                                                 
Total params: 16,764
Trainable params: 16,764
Non-trainable params: 0
_________________________________________________________________


In [8]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=-1),keepdims=True)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [9]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(X_train)

14/14 [==============================] - 1s 3ms/step - loss: 2.7479 - reconstruction_loss: -2.2175 - kl_loss: 0.5319


In [10]:
vae.predict(X_test)

NotImplementedError: in user code:

    File "/Users/chinmayasukumar/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/chinmayasukumar/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/chinmayasukumar/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/chinmayasukumar/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/chinmayasukumar/opt/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/chinmayasukumar/opt/anaconda3/li